In [2]:
from langchain.llms import GooglePalm
from keys import google_api_key
llm = GooglePalm(google_api_key=google_api_key,temperature=0.2)
res=llm("how oceans produce more o2 than trees")
print(res)

C:\Users\vivek\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\vivek\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


The ocean produces more oxygen than trees because it is much larger and has a greater surface area. The ocean covers over 70% of the Earth's surface, while forests only cover about 30%. This means that the ocean has a much greater capacity to absorb carbon dioxide and produce oxygen.

In addition, the ocean is home to a vast array of phytoplankton, which are tiny plants that produce oxygen through photosynthesis. Phytoplankton are responsible for producing about half of the oxygen in the Earth's atmosphere. Trees also produce oxygen through photosynthesis, but they only account for about 20% of the oxygen in the atmosphere.

So, while trees are important contributors to the oxygen cycle, the ocean is the single largest source of oxygen on Earth.


In [3]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
db_user = "root"
db_password = "vivek123"
db_host = "localhost"
db_name = "k_tshirts"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)

print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

In [4]:
from langchain_experimental.sql import SQLDatabaseChain
db_chain=SQLDatabaseChain.from_llm(llm,db,verbose=True)


In [5]:
q1= db_chain.run("what are different brand t-shirts available")



> Entering new SQLDatabaseChain chain...
what are different brand t-shirts available
SQLQuery:

C:\Users\vivek\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


SELECT DISTINCT brand FROM t_shirts
SQLResult: [('Van Huesen',), ('Levi',), ('Nike',), ('Adidas',)]
Answer:Van Huesen, Levi, Nike, Adidas
> Finished chain.


In [6]:
q2=db_chain.run("display the total quantity of white Nike t-shirts available")



> Entering new SQLDatabaseChain chain...
display the total quantity of white Nike t-shirts available
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' AND color = 'White'
SQLResult: [(30,), (57,)]
Answer:30
> Finished chain.


In [7]:
q3=db_chain.run("select sum(stock_quantity) where branc='Nike' and color='White';")



> Entering new SQLDatabaseChain chain...
select sum(stock_quantity) where branc='Nike' and color='White';
SQLQuery:SELECT SUM(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White'
SQLResult: [(Decimal('87'),)]
Answer:87
> Finished chain.


In [8]:
db_chain.run("what are the different color t-shirts available in each brand")



> Entering new SQLDatabaseChain chain...
what are the different color t-shirts available in each brand
SQLQuery:SELECT brand, color FROM t_shirts GROUP BY brand, color ORDER BY brand, color
SQLResult: [('Van Huesen', 'Red'), ('Van Huesen', 'Blue'), ('Van Huesen', 'Black'), ('Van Huesen', 'White'), ('Levi', 'Red'), ('Levi', 'Blue'), ('Levi', 'Black'), ('Levi', 'White'), ('Nike', 'Red'), ('Nike', 'Blue'), ('Nike', 'Black'), ('Nike', 'White'), ('Adidas', 'Red'), ('Adidas', 'Blue'), ('Adidas', 'Black'), ('Adidas', 'White')]
Answer:Van Huesen: Red, Blue, Black, White
Levi: Red, Blue, Black, White
Nike: Red, Blue, Black, White
Adidas: Red, Blue, Black, White
> Finished chain.


'Van Huesen: Red, Blue, Black, White\nLevi: Red, Blue, Black, White\nNike: Red, Blue, Black, White\nAdidas: Red, Blue, Black, White'

In [13]:
q3=db_chain.run("select sum(stock_quantity) where brand='Nike' and color='White';")
q5=db_chain.run("select sum(stock_quantity) where brand='Nike' and color='Red';")
q6=db_chain.run("select sum(stock_quantity) where brand='Nike' and color='Black';")
q7=db_chain.run("select sum(stock_quantity) where brand='Adidas' and color='Black';")



> Entering new SQLDatabaseChain chain...
select sum(stock_quantity) where brand='Nike' and color='White';
SQLQuery:SELECT SUM(stock_quantity) FROM t_shirts WHERE brand='Nike' AND color='White'
SQLResult: [(Decimal('87'),)]
Answer:87
> Finished chain.


> Entering new SQLDatabaseChain chain...
select sum(stock_quantity) where brand='Nike' and color='Red';
SQLQuery:SELECT SUM(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'Red'
SQLResult: [(Decimal('136'),)]
Answer:136
> Finished chain.


> Entering new SQLDatabaseChain chain...
select sum(stock_quantity) where brand='Nike' and color='Black';
SQLQuery:SELECT SUM(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'Black'
SQLResult: [(Decimal('179'),)]
Answer:179
> Finished chain.


> Entering new SQLDatabaseChain chain...
select sum(stock_quantity) where brand='Adidas' and color='Black';
SQLQuery:SELECT SUM(stock_quantity) FROM t_shirts WHERE brand = 'Adidas' AND color = 'Black'
SQLResult: [(Decimal('235'),)]

In [15]:
# llm will be confused when the question is not clear and give the wrong results
# for that reason we use few shot learning where question and answer are specified
# we need to save as a dictonary

few_shots = [
    {"Question": "How many white Nike t-shirts are available",
     "SQLQuery": "select sum(stock_quantity) from t_shirts where color='White' and brand='Nike';",
     'SQLResult':"Result of SQL query",
      'Answer': q3
     },
     {
      "Question": "How many Red Nike t-shirts are available",
     "SQLQuery": "select sum(stock_quantity) from t_shirts where color='Red' and brand='Nike';",
     'SQLResult':"Result of SQL query",
      "Answer": q5
     },
     {
     "Question": "How many Black Nike t-shirts are available",
     "SQLQuery": "select sum(stock_quantity) from t_shirts where color='Black' and brand='Nike';",
     'SQLResult':"Result of SQL query",
      "Answer": q6   
     },
     {
     "Question": "How many Black Adidas t-shirts are available",
     "SQLQuery": "select sum(stock_quantity) from t_shirts where color='Black' and brand='Adidas';",
     'SQLResult':"Result of SQL query",
      "Answer": q7
     }
]


In [16]:
to_join=[" ".join(x.values())for x in few_shots]
to_join

["How many white Nike t-shirts are available select sum(stock_quantity) from t_shirts where color='White' and brand='Nike'; Result of SQL query 87",
 "How many Red Nike t-shirts are available select sum(stock_quantity) from t_shirts where color='Red' and brand='Nike'; Result of SQL query 136",
 "How many Black Nike t-shirts are available select sum(stock_quantity) from t_shirts where color='Black' and brand='Nike'; Result of SQL query 179",
 "How many Black Adidas t-shirts are available select sum(stock_quantity) from t_shirts where color='Black' and brand='Adidas'; Result of SQL query 235"]

In [17]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

C:\Users\vivek\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [18]:
from langchain.vectorstores import Chroma
vectorstore=Chroma.from_texts(to_join,embedding=embeddings,metadatas=few_shots)

In [20]:
from langchain.prompts import SemanticSimilarityExampleSelector
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

example_selector.select_examples({"Question": "How many white Adidas t-shirts are available?"})

[{'Answer': '235',
  'Question': 'How many Black Adidas t-shirts are available',
  'SQLQuery': "select sum(stock_quantity) from t_shirts where color='Black' and brand='Adidas';",
  'SQLResult': 'Result of SQL query'},
 {'Answer': '87',
  'Question': 'How many white Nike t-shirts are available',
  'SQLQuery': "select sum(stock_quantity) from t_shirts where color='White' and brand='Nike';",
  'SQLResult': 'Result of SQL query'}]

In [21]:
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [22]:
### my sql based instruction prompt
mysql_prompt = """You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: Query to run with no pre-amble
SQLResult: Result of the SQLQuery
Answer: Final answer here

No pre-amble.
"""

In [23]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [24]:
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"], #These variables are used in the prefix and suffix
)

In [25]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)

In [30]:
new_chain("How many Blue extras small Levi t-shirts are available")



> Entering new SQLDatabaseChain chain...
How many Blue extras small Levi t-shirts are available
SQLQuery:select sum(stock_quantity) from t_shirts where color='Blue' and size='XS' and brand='Levi';
SQLResult: [(Decimal('22'),)]
Answer:22
> Finished chain.


{'query': 'How many Blue extras small Levi t-shirts are available',
 'result': '22'}

In [31]:
new_chain("how many black Adidas t-shirts are available")



> Entering new SQLDatabaseChain chain...
how many black Adidas t-shirts are available
SQLQuery:select sum(stock_quantity) from t_shirts where color='Black' and brand='Adidas';
SQLResult: [(Decimal('235'),)]
Answer:235
> Finished chain.


{'query': 'how many black Adidas t-shirts are available', 'result': '235'}

In [35]:
new_chain("select the total revenue that can be generated by selling nike")



> Entering new SQLDatabaseChain chain...
select the total revenue that can be generated by selling nike
SQLQuery:select sum(price*stock_quantity) from t_shirts where brand='Nike';
SQLResult: [(Decimal('21516'),)]
Answer:21516
> Finished chain.


{'query': 'select the total revenue that can be generated by selling nike',
 'result': '21516'}

In [38]:
new_chain("what is the costliest tshirt you have in your store and what is price of it along with its brand")



> Entering new SQLDatabaseChain chain...
what is the costliest tshirt you have in your store and what is price of it along with its brand
SQLQuery:select brand, price from t_shirts order by price desc limit 1;
SQLResult: [('Adidas', 49)]
Answer:Adidas 49
> Finished chain.


{'query': 'what is the costliest tshirt you have in your store and what is price of it along with its brand',
 'result': 'Adidas 49'}